# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# List all collections in the 'uk_food' database
collections = db.list_collection_names()
print("Collections in the 'uk_food' database:")
for collection in collections:
    print(collection)


Collections in the 'uk_food' database:
establishments


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {"HygieneScore": 20}
results = establishments.find(query)

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of documents with hygiene score 20: {count}")

# Display the first document in the results using pprint
from pprint import pprint
results = establishments.find(query)

first_result = next(results, None)
if first_result:
    pprint(first_result)
else:
    print("No documents found with hygiene score 20.")

Number of documents with hygiene score 20: 0
No documents found with hygiene score 20.


In [7]:
# Convert the result to a Pandas DataFrame
df_results = pd.DataFrame(list(results))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df_results)}")

# Display the first 10 rows of the DataFrame
print(df_results.head(10))

Number of rows in the DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    "LocalAuthorityName": {"$regex": "London"},
    "RatingValue": {"$gte": 4}
}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of documents with RatingValue >= 4 in London: {count}")

# Display the first document in the results using pprint
results = establishments.find(query)

# Covert to a list to help visulize results
results_list = list(results)
if results_list:
    pprint(results_list[0])
else:
    print("No documents found.")


Number of documents with RatingValue >= 4 in London: 33
{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66a1b7a7a291c8a8dc9131d2'),
 'geocode': {'latitude': '51.369321', 'longitude': '0.508551'},
 'links': [{'href': 'https://api.ratings.foo

In [9]:
# Convert the result to a Pandas DataFrame
df_results = pd.DataFrame(list(results))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df_results)}")

# Display the first 10 rows of the DataFrame
print(df_results.head(10))

Number of rows in the DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [34]:
import json
import pandas as pd
from geopy.distance import great_circle

# Load the establishments data from the JSON file
file_path = '/Users/mauriciogil/Desktop/nosql-challenge/Resources/establishments.json'
with open(file_path, 'r') as file:
    establishments_list = json.load(file)

# Convert to Pandas DataFrame
df = pd.DataFrame(establishments_list)

# Retrieve latitude and longitude of "Penang Flavours"
penang_flavours = df[df['BusinessName'] == "Penang Flavours"].iloc[0]
penang_lat = float(penang_flavours['geocode']['latitude'])
penang_lon = float(penang_flavours['geocode']['longitude'])

# Convert hygiene scores from JSON strings to dictionaries
df['scores'] = df['scores'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Filter for establishments with RatingValue of 5
filtered_df = df[df['RatingValue'] == 5]

# Calculate distance from "Penang Flavours"
def calculate_distance(row):
    lat = float(row['geocode']['latitude'])
    lon = float(row['geocode']['longitude'])
    return great_circle((penang_lat, penang_lon), (lat, lon)).kilometers

filtered_df['Distance'] = filtered_df.apply(calculate_distance, axis=1)

# Sort by hygiene score (assuming 'Hygiene' is a key in the scores dictionary)
filtered_df['HygieneScore'] = filtered_df['scores'].apply(lambda x: x.get('Hygiene', float('inf')))
sorted_df = filtered_df.sort_values(by=['Distance', 'HygieneScore'])

# Get the top 5 establishments
top_5_establishments = sorted_df.head(5)

print(top_5_establishments[['BusinessName', 'RatingValue', 'Distance', 'HygieneScore']])


                                            BusinessName  RatingValue  \
33231                                     Kitchen Garden          5.0   
33219                                   Black Eye Coffee          5.0   
33184                                  Hardys Free house          5.0   
33186                                           Zaibatsu          5.0   
33192  The Trafalgar also T/A Balboa Colombian Pacifi...          5.0   

       Distance  HygieneScore  
33231  0.432985           0.0  
33219  0.460225           5.0  
33184  0.463786           5.0  
33186  0.479258           5.0  
33192  0.482788           0.0  


/var/folders/3r/0zp_mxd52m7gvcpxj9t957hm0000gn/T/ipykernel_90246/3294397510.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Distance'] = filtered_df.apply(calculate_distance, axis=1)
/var/folders/3r/0zp_mxd52m7gvcpxj9t957hm0000gn/T/ipykernel_90246/3294397510.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['HygieneScore'] = filtered_df['scores'].apply(lambda x: x.get('Hygiene', float('inf')))


In [40]:
# Convert result to Pandas DataFrame
filtered_df = pd.DataFrame(results_list)

# Display the DataFrame
print(filtered_df[['BusinessName', 'RatingValue', 'Distance', ]])

                            BusinessName  RatingValue     Distance
0                              Charlie's            4  4627.439468
1                Mv City Cruises Erasmus            5  4627.873179
2              Benfleet Motor Yacht Club            4  4631.968418
3   Coombs Catering t/a The Lock and Key            5  4636.846754
4               Tilbury Seafarers Centre            5  4636.847650
5                             Mv Valulla            5  4640.460834
6                          Tereza Joanne            5  4648.301822
7                       Brick Lane Brews            4  4649.173485
8          The Nuance Group (UK) Limited            5  4649.188826
9                               WH Smith            5  4649.188850
10                      City Bar & Grill            5  4649.188850
11                            Caffè Nero            5  4649.485929
12                            Jet Centre            5  4649.485870
13                Mv Sunborn Yacht Hotel            5  4650.09

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [41]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results
from pymongo import MongoClient

# Create a MongoClient instance and connect to the 'uk_food' database
mongo = MongoClient(port=27017)
db = mongo['uk_food']


pipeline = [
    # Match documents with a hygiene score of 0
    {"$match": {"scores.Hygiene": 0}},
    
    # Group by Local Authority and count the number of establishments
    {"$group": {
        "_id": "$LocalAuthorityName",
        "count": {"$sum": 1}
    }},
    
    # Sort by count in descending order
    {"$sort": {"count": -1}}
]

# Execute the aggregation pipeline
results = list(db.establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"Number of documents in the result: {len(results)}")

# Print the first 10 results
for result in results[:10]:
    print(result)


Number of documents in the result: 55
{'_id': 'Thanet', 'count': 1130}
{'_id': 'Greenwich', 'count': 882}
{'_id': 'Maidstone', 'count': 713}
{'_id': 'Newham', 'count': 711}
{'_id': 'Swale', 'count': 686}
{'_id': 'Chelmsford', 'count': 680}
{'_id': 'Medway', 'count': 672}
{'_id': 'Bexley', 'count': 607}
{'_id': 'Southend-On-Sea', 'count': 586}
{'_id': 'Tendring', 'count': 542}


In [42]:
# Create a MongoClient instance and connect to the 'uk_food' database
mongo = MongoClient(port=27017)
db = mongo['uk_food']

# Define the aggregation pipeline
pipeline = [
    # Match documents with a hygiene score of 0
    {"$match": {"scores.Hygiene": 0}},
    
    # Group by Local Authority and count the number of establishments
    {"$group": {
        "_id": "$LocalAuthorityName",
        "count": {"$sum": 1}
    }},
    
    # Sort by count in descending order
    {"$sort": {"count": -1}}
]

# Execute the aggregation pipeline
results = list(db.establishments.aggregate(pipeline))

# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Rename columns for clarity
df.columns = ['LocalAuthorityName', 'EstablishmentCount']

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows in the DataFrame: 55
  LocalAuthorityName  EstablishmentCount
0             Thanet                1130
1          Greenwich                 882
2          Maidstone                 713
3             Newham                 711
4              Swale                 686
5         Chelmsford                 680
6             Medway                 672
7             Bexley                 607
8    Southend-On-Sea                 586
9           Tendring                 542
